In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "So0xTl9tCAdXqx4JniGO"),
    ca_certs="http_ca.crt",
)

client_info = es.info()

print("connected to elasticsearch")
pprint(client_info.body)


connected to elasticsearch
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'Ptp3XLACR3-Wmx0PcdxoZA',
 'name': 'c899a61c3425',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-08-24T22:05:04.526302670Z',
             'build_flavor': 'default',
             'build_hash': '0c781091a2f57de895a73a1391ff8426c0153c8d',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.2.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.1.3'}}


In [2]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index")


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [5]:
# !pip install tqdm
import json
from tqdm import tqdm

document_ids = []
dummy_data = json.load(open("./data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index="my_index", body=document)
    document_ids.append(response.body["_id"])

100%|██████████| 3/3 [00:00<00:00, 18.75it/s]


In [6]:
document_ids

['dJUaTZkBza3mxyTRqKV3', 'dZUaTZkBza3mxyTRqKWr', 'dpUaTZkBza3mxyTRqKXf']

# Delete operation

In [7]:
response = es.delete(index="my_index", id=document_ids[0])

In [8]:
from pprint import pprint
pprint(response.body)

{'_id': 'dJUaTZkBza3mxyTRqKV3',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 6,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'deleted'}


In [9]:
try:
    response = es.delete(index="my_index", id="nonexisting_id")
except Exception as e:
    print("Error:", e)

Error: NotFoundError(404, "{'_index': 'my_index', '_id': 'nonexisting_id', '_version': 1, 'result': 'not_found', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7, '_primary_term': 1}")
